In [ ]:
import numpy as np
import pandas as pd
import warnings
import joblib
import time
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import ExtraTreesRegressor
import optuna
import torch

RANDOM_SEED = 0

warnings.filterwarnings("ignore")
optuna.logging.set_verbosity(optuna.logging.WARNING)
np.random.seed(RANDOM_SEED)

# =============================================================================
# GPU CONFIGURATION
# =============================================================================
print("="*80)
print("EXTRATREES HYPERPARAMETER OPTIMIZATION")
print("="*80)

# Check CPU cores for parallelization
import os
N_CORES = os.cpu_count()
print(f"✓ CPU cores available: {N_CORES}")

# =============================================================================
# LOAD PREPROCESSED DATA
# =============================================================================
print("\n" + "="*80)
print("LOADING PREPROCESSED DATA")
print("="*80)

X_train = joblib.load('../../data/preprocessed/X_train.pkl')
y_train = joblib.load('../../data/preprocessed/y_train.pkl')
X_test = joblib.load('../../data/preprocessed/X_test.pkl')
metadata = joblib.load('../../data/preprocessed/metadata.pkl')

print(f"✓ Train shape: {X_train.shape}")
print(f"✓ Target shape: {y_train.shape}")
print(f"✓ Test shape: {X_test.shape}")

# =============================================================================
# OPTIMIZATION SETTINGS
# =============================================================================
FAST_MODE = True  # Set to False for more thorough search
N_TRIALS = 15 if FAST_MODE else 400
N_FOLDS = 5

print(f"\n{'='*60}")
print(f"OPTIMIZATION SETTINGS")
print(f"{'='*60}")
print(f"Mode: {'FAST' if FAST_MODE else 'THOROUGH'}")
print(f"Trials: {N_TRIALS}")
print(f"Folds: {N_FOLDS}")
print(f"Parallel Optuna jobs: {N_CORES}")

# =============================================================================
# OBJECTIVE FUNCTION
# =============================================================================
def objective_extratrees(trial, X, y):
    """Objective function for ExtraTrees optimization"""
    
    # Hyperparameters to optimize
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'max_depth': trial.suggest_int('max_depth', 10, 50),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
        'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2', None]),
        'bootstrap': trial.suggest_categorical('bootstrap', [True, False]),
        'random_state': RANDOM_SEED,
        'n_jobs': 2,  # Use all cores for fitting each model
    }

    kf = KFold(n_splits=N_FOLDS, shuffle=True, random_state=RANDOM_SEED)
    scores = []

    for fold_idx, (train_idx, val_idx) in enumerate(kf.split(X)):
        X_tr, X_val = X.iloc[train_idx], X.iloc[val_idx]
        
        # Handle both pandas Series and numpy arrays
        if isinstance(y, pd.Series):
            y_tr, y_val = y.iloc[train_idx], y.iloc[val_idx]
        else:
            y_tr, y_val = y[train_idx], y[val_idx]

        model = ExtraTreesRegressor(**params)
        model.fit(X_tr, y_tr)

        preds = np.clip(model.predict(X_val), 0, 100)
        rmse = np.sqrt(mean_squared_error(y_val, preds))
        scores.append(rmse)

    return np.mean(scores)

# =============================================================================
# RUN OPTIMIZATION
# =============================================================================
print("\n" + "="*80)
print("STARTING BAYESIAN OPTIMIZATION")
print("="*80)

start_time = time.time()

study = optuna.create_study(
    direction='minimize',
    sampler=optuna.samplers.TPESampler(seed=RANDOM_SEED)
)

study.optimize(
    lambda trial: objective_extratrees(trial, X_train, y_train),
    n_trials=N_TRIALS,
    show_progress_bar=True,
    n_jobs=N_CORES
)

optimization_time = time.time() - start_time

# =============================================================================
# RESULTS
# =============================================================================
print("\n" + "="*80)
print("OPTIMIZATION RESULTS")
print("="*80)
print(f"Best RMSE: {study.best_value:.6f}")
print(f"Optimization time: {optimization_time:.1f}s")
print(f"\nBest parameters:")
for param, value in sorted(study.best_params.items()):
    print(f"  {param:20s}: {value}")

# =============================================================================
# SAVE RESULTS
# =============================================================================
# Save best parameters
best_params = study.best_params.copy()
best_params.update({
    'random_state': RANDOM_SEED,
    'n_jobs': -1
})

joblib.dump(best_params, 'extratrees_params.pkl')
print("\n✓ Parameters saved to: extratrees_params.pkl")

# Save optimization history
history = pd.DataFrame({
    'trial': [t.number for t in study.trials],
    'value': [t.value for t in study.trials],
    'params': [str(t.params) for t in study.trials]
})
history.to_csv('extratrees_history.csv', index=False)
print("✓ History saved to: extratrees_history.csv")

# Save study object
joblib.dump(study, 'extratrees_study.pkl')
print("✓ Study saved to: extratrees_study.pkl")

# Save summary
summary = {
    'model': 'ExtraTrees',
    'best_rmse': study.best_value,
    'n_trials': N_TRIALS,
    'n_folds': N_FOLDS,
    'optimization_time': optimization_time,
    'best_params': best_params
}
joblib.dump(summary, 'extratrees_summary.pkl')
print("✓ Summary saved to: extratrees_summary.pkl")

# =============================================================================
# TRAIN FINAL MODEL ON FULL TRAINING DATA
# =============================================================================
print("\n" + "="*80)
print("TRAINING FINAL MODEL ON FULL DATASET")
print("="*80)

final_model = ExtraTreesRegressor(**best_params)
final_model.set_params(verbose=1)  # Show training progress

print("Training final model...")
final_model.fit(X_train, y_train)

# Save final model
joblib.dump(final_model, 'extratrees_final_model.pkl')
print("\n✓ Final model saved to: extratrees_final_model.pkl")

# =============================================================================
# GENERATE PREDICTIONS AND SUBMISSION FILE
# =============================================================================
print("\n" + "="*80)
print("GENERATING PREDICTIONS")
print("="*80)

# Make predictions on test set
test_predictions = final_model.predict(X_test)

# Clip predictions to valid range [0, 100]
test_predictions = np.clip(test_predictions, 0, 100)

# Create submission DataFrame
submission = pd.DataFrame({
    'id': range(len(test_predictions)),
    'exam_score': test_predictions
})

# Save submission file
submission.to_csv('submission.csv', index=False)
print("✓ Submission file saved to: submission.csv")

print(f"\nSubmission statistics:")
print(f"  Min prediction: {test_predictions.min():.2f}")
print(f"  Max prediction: {test_predictions.max():.2f}")
print(f"  Mean prediction: {test_predictions.mean():.2f}")
print(f"  Median prediction: {np.median(test_predictions):.2f}")
print(f"  Std prediction: {test_predictions.std():.2f}")
print(f"  Shape: {submission.shape}")

# =============================================================================
# FEATURE IMPORTANCE
# =============================================================================
print("\n" + "="*80)
print("FEATURE IMPORTANCE")
print("="*80)

# Get feature importance
feature_importance = pd.DataFrame({
    'feature': X_train.columns,
    'importance': final_model.feature_importances_
}).sort_values('importance', ascending=False)

print("\nTop 10 most important features:")
print(feature_importance.head(10).to_string(index=False))

# Save feature importance
feature_importance.to_csv('extratrees_feature_importance.csv', index=False)
print("\n✓ Feature importance saved to: extratrees_feature_importance.csv")

# =============================================================================
# MODEL STATISTICS
# =============================================================================
print("\n" + "="*80)
print("MODEL STATISTICS")
print("="*80)
print(f"Number of trees: {best_params['n_estimators']}")
print(f"Max depth: {best_params['max_depth']}")
print(f"Total number of leaves: {sum(tree.tree_.n_leaves for tree in final_model.estimators_)}")
print(f"Average leaves per tree: {sum(tree.tree_.n_leaves for tree in final_model.estimators_) / len(final_model.estimators_):.1f}")

# =============================================================================
# FINAL SUMMARY
# =============================================================================
print("\n" + "="*80)
print("EXTRATREES OPTIMIZATION COMPLETE!")
print("="*80)
print(f"✓ Best CV RMSE: {study.best_value:.6f}")
print(f"✓ Final model trained on {len(X_train)} samples")
print(f"✓ Model has {best_params['n_estimators']} trees")
print(f"✓ Predictions generated for {len(X_test)} test samples")
print(f"✓ All results saved")
print("\nFiles created:")
print("  • extratrees_params.pkl")
print("  • extratrees_history.csv")
print("  • extratrees_study.pkl")
print("  • extratrees_summary.pkl")
print("  • extratrees_final_model.pkl")
print("  • extratrees_feature_importance.csv")
print("  • submission.csv")
print("="*80)